In [1]:
import os
import time
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
import shutil
path=''

In [2]:
from dask.distributed import Client
client = Client(n_workers=3, processes=True)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 12,Total memory: 11.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50434,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 11.92 GiB
Comm: tcp://127.0.0.1:50451,Total threads: 4
Dashboard: http://127.0.0.1:50454/status,Memory: 3.97 GiB
Nanny: tcp://127.0.0.1:50437,


In [3]:
contenido = os.listdir(path)
contenido

['example0.csv',
 'example1.csv',
 'example10.csv',
 'example2.csv',
 'example3.csv',
 'example4.csv',
 'example5.csv',
 'example6.csv',
 'example7.csv',
 'example8.csv',
 'example9.csv']

In [4]:
df_union=pd.DataFrame()
archivos = os.listdir(path)
column_name = ['col5_float','col7_float','col9_float','col11_float','col13_float','col15_float','col17_float','col19_float','col21_float','col23_float','col25_float','col27_float','target_boolean','col31_integer','col33_integer','col35_integer']
valColumns = {} 

In [5]:
len(column_name)

16

In [ ]:
''' 
for i in range(5,len(l),2):
    #print(i, i+1)
'''

In [41]:
@dask.delayed
def getValids(path):
    df = pd.read_csv(path)
    columns = list(df.columns)
    
    colL = []
    for c in range(5,len(columns),2):
        #cont = 5 + c - 1 
        std1=df[columns[c]].describe()
        std2=df[columns[c+1]].describe()
        if(std1[2]>std2[2]):
            colL.append(columns[c+1])
        else:
            colL.append(columns[c])
    #valColumns[filename] = 
    return colL

In [42]:
''' 
results =[]
for ar in archivos:
        results.append(getValids(path + '\\' + ar))
results
'''
for ar in archivos:
        valColumns[ar] = getValids(path + '\\' + ar)
valColumns

{'example0.csv': Delayed('getValids-38ff7d88-1ad8-4767-af3f-6132d2c6a594'),
 'example1.csv': Delayed('getValids-557ac018-ae3d-41a9-89c7-a3880d8b3dfb'),
 'example10.csv': Delayed('getValids-cebab9a7-d8d1-4451-81ce-92a56f7cffeb'),
 'example2.csv': Delayed('getValids-5ba7a6ba-c0ca-4b15-8e73-c2e0201db892'),
 'example3.csv': Delayed('getValids-c5357843-6f94-466d-8013-2d1a82f456b1'),
 'example4.csv': Delayed('getValids-55e64364-319d-4cf8-800f-ed852f299f15'),
 'example5.csv': Delayed('getValids-503055aa-b6ac-4022-b11d-b7facf9f247f'),
 'example6.csv': Delayed('getValids-43c673e9-409b-4164-8272-48dfb7035b86'),
 'example7.csv': Delayed('getValids-01a5c6d9-29bf-41de-9dc5-2789f2968115'),
 'example8.csv': Delayed('getValids-c88c2c34-3ed7-49c8-baaf-3c425f62f3b3'),
 'example9.csv': Delayed('getValids-fd24a436-fe2e-45b1-8437-9b37d6831446')}

In [43]:
res = dask.compute(valColumns)

In [44]:
res = res[0]

In [45]:
res['example0.csv']

['col6_float',
 'col8_float',
 'col10_float',
 'col12_float',
 'col13_float',
 'col15_float',
 'col17_float',
 'col20_float',
 'col22_float',
 'col23_float',
 'col26_float',
 'col27_float',
 'target_boolean',
 'col31_integer',
 'col33_integer',
 'col36_integer']

In [46]:
dropList = ['col1_boolean','col2_boolean','col3_boolean','col4_boolean']
renameLst = ['time','col5_float','col7_float','col9_float','col11_float','col13_float','col15_float','col17_float','col19_float','col21_float','col23_float','col25_float','col27_float','target_boolean','col31_integer','col33_integer','col35_integer','Fname']  
wpath= ''

@dask.delayed
def modFiles(path, wpath,fname, lst):
    df = pd.read_csv(path+fname)
    df['File'] = fname
    lst = dropList + lst
    df = df.drop(columns=lst)
    df.columns = renameLst
    wpath = wpath + fname
    df.to_csv(wpath)
    
    

In [47]:
procLst = []
for ar, lst in res.items():
    procLst.append(modFiles(path, wpath, ar, lst))
procLst

[Delayed('modFiles-d8a2c627-8eb9-4f87-b121-054feae84d95'),
 Delayed('modFiles-14b6e4b3-7c38-468a-b39c-8ccdbc64cfab'),
 Delayed('modFiles-51a5b563-a929-45a4-9945-83dca5a0026a'),
 Delayed('modFiles-0d64b760-0bd3-41f5-992d-6c7dccd32ed0'),
 Delayed('modFiles-ca864a1d-6a1f-43c0-94ac-b473ef601f16'),
 Delayed('modFiles-e18f1c90-753e-4751-91cd-23dc412cf2fa'),
 Delayed('modFiles-f72a3a91-8e77-43b0-9270-a44adb8731c8'),
 Delayed('modFiles-93071aef-6bf6-44ad-b443-06b7b7e73778'),
 Delayed('modFiles-7f23100b-6abd-4eff-a5db-03a5fe15bca6'),
 Delayed('modFiles-07e29beb-b6b1-459b-b7ac-6685bba7dedb'),
 Delayed('modFiles-62827b57-163e-42ed-b7c1-ad44c1f4dcbd')]

In [48]:
dask.compute(procLst)

([None, None, None, None, None, None, None, None, None, None, None],)

In [49]:
#df = pd.read_csv(path + '\\example0.csv')
#df['arname'] = 'example0.csv'
#df

In [50]:
client.close()